In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pfbmod import forward_pfb as pfb
from scipy.interpolate import CubicSpline

In [ ]:
def mycorr(a,b):
    myxcorr=np.zeros(len(a),dtype=a.dtype)
    print("len is", len(a))
    for i in range(len(a)):
        s=0
        for j in range(len(a)):
            s+=a[j]*np.conj(b[j-i])
        myxcorr[i]=s
    return myxcorr

In [10]:
4096*1.6

6553.6

In [23]:
N=4096
bigN=4096*100
np.random.seed(42)
n=np.arange(0,bigN)
k1=1835.2
k2=1845.1
delay=-6560
# print(delay%(2*np.pi))
d=200000
dk = 0.02 # BW of 6 kHz
y=np.sinc(dk*(n-d)/N)*np.cos(2*np.pi*k1*(n-d)/N)+np.sinc(dk*(n-d)/N)*np.cos(2*np.pi*k2*(n-d)/N)+0.2*np.random.randn(len(n))
y2=np.sinc(dk*(n-delay-d)/N)*np.cos(2*np.pi*k1*(n-delay-d)/N)+np.sinc(dk*(n-delay-d)/N)*np.cos(2*np.pi*k2*(n-delay-d)/N)+0.2*np.random.randn(len(n))


# z=np.zeros(2*N,dtype=y.dtype)
# z2=np.zeros(2*N,dtype=y.dtype)
# z[:N]=y
# z2[:N]=y2
# ft = np.fft.fft(z)
# ft2=np.fft.fft(z2)

# k=510
# y=np.cos(2*np.pi*k*n/N)+np.random.randn(N)
# y2=np.cos(2*np.pi*k*n/N - dphase)+np.random.randn(N)
# ft1=np.fft.fft(y)
# ft3=np.fft.fft(y2)


In [ ]:
plt.plot(y2.real,label='y2')
plt.plot(y,label='y')# plt.xlim(0,200)
plt.legend()

In [24]:
yy=y.reshape(-1,4096)
yy2=y2.reshape(-1,4096)
f1=np.fft.rfft(yy,axis=1)
f2=np.fft.rfft(yy2,axis=1)
# f1=pfb(y)
# f2=pfb(y2)

In [ ]:
xcorrtemp=f1*np.conj(f2)

In [ ]:
plt.imshow(np.log10(np.abs(xcorrtemp)),aspect='auto')
# plt.xlim(1800,1900)

In [ ]:
plt.plot(np.abs(f1[51,:]),'-*')
np.argmax(np.abs(f1[51,:]))
plt.xlim(1800,1900)

In [ ]:
## Correlate the whole thing

a=np.zeros(2*len(y),dtype=y.dtype)
a2=np.zeros(2*len(y2),dtype=y.dtype)
a[:len(y)]=y.copy()
a2[:len(y2)]=y2.copy()
xcorrtime1=np.fft.irfft(np.fft.rfft(a)*np.conj(np.fft.rfft(a2)))

In [ ]:
plt.plot(xcorrtime1)
m=np.argmax(xcorrtime1)
print(m)
plt.xlim(m-500,m+500)

In [ ]:
plt.plot(xcorrtime1[::4096])
m=np.argmax(xcorrtime1[::4096])
print(m)
# plt.xlim(m-10,m+10)

In [25]:
Nsmall=f1.shape[0]
print(Nsmall)
xcorrtime2=np.zeros((2*Nsmall,2),dtype='complex128')
for i,chan in enumerate([1835,1845]):
    p0=np.zeros(2*Nsmall,dtype=f1.dtype)
    p1=np.zeros(2*Nsmall,dtype=f2.dtype)
    p0[:Nsmall]=f1[:Nsmall,chan].flatten()
    p1[:Nsmall]=f2[:Nsmall,chan].flatten()
    ft1=np.fft.fft(p0)
    ft2=np.fft.fft(p1)
    xcorr=ft1*np.conj(ft2)
    xcorrtime2[:,i]=np.fft.ifft(xcorr)

100


In [26]:
xcorr2sum = np.sum(xcorrtime2,axis=1)

In [28]:
np.argmax(np.abs(xcorr2sum))

1

In [ ]:
plt.plot(np.abs(xcorr2sum))

In [ ]:
amp=np.sum(np.real(xcorr2sum)*xcorrtime1[::4096])/np.sum(np.real(xcorr2sum)**2)

In [ ]:
f=plt.gcf()
f.set_size_inches(10,4)

plt.plot(xcorrtime1[::4096],'-o',label='raw data xcorr sampled every 4096 sample')
plt.plot(amp*np.real(xcorr2sum),'-*',label='FFTed channel xcorr, real part')
plt.legend()
plt.title('Forward problem')
plt.xlabel('spec num')

In [ ]:
f=plt.gcf()
f.set_size_inches(10,4)
plt.suptitle('phase of coarse xcorr with time')
plt.subplot(121)
plt.plot(np.angle(xcorrtime2[:,0]),'-*')
plt.subplot(122)
plt.plot(np.unwrap(np.angle(xcorrtime2[:,0])))
plt.title('unwrapped')

# plt.xlim(40,90)

In [ ]:
y=np.unwrap(np.angle(xcorrtime2[40:90,0]))
m,c=np.polyfit(np.arange(0,len(y))*4096,y,1)
print(m*4096/(2*np.pi))

In [ ]:
y=np.unwrap(np.angle(xcorrtime2[:50,0]))
m,c=np.polyfit(np.arange(0,len(y))*4096,y,1)
print(m*4096/(2*np.pi))
x=np.arange(0,xcorrtime2.shape[0]*4096)
newphase=(m+2*np.pi*1835/4096)*x+c#-2*np.pi*1150*60
cs=CubicSpline(np.arange(0,xcorrtime2.shape[0])*4096,np.abs(xcorrtime2[:,0]))
newmag=cs(x)
# newmag = np.interp(x,np.arange(0,len(y))*4096,np.abs(c1full))
zz1=newmag*np.exp(1J*newphase)

In [ ]:
y=np.unwrap(np.angle(xcorrtime2[:50,1]))
m,c=np.polyfit(np.arange(0,len(y))*4096,y,1)
print(m*4096/(2*np.pi))
x=np.arange(0,xcorrtime2.shape[0]*4096)
newphase=(m+2*np.pi*1845/4096)*x+c#-2*np.pi*1150*60
cs=CubicSpline(np.arange(0,xcorrtime2.shape[0])*4096,np.abs(xcorrtime2[:,1]))
newmag=cs(x)
# newmag = np.interp(x,np.arange(0,len(y))*4096,np.abs(c1full))
zz2=newmag*np.exp(1J*newphase)

In [ ]:
plt.plot(np.real(zz1))
plt.plot(np.real(zz2))
m=31000
plt.xlim(m-100,m+100)

In [ ]:
plt.plot(np.real(zz1)+np.real(zz2))
m=31000
plt.xlim(m-4000,m+4000)

In [ ]:
np.argmax()

In [ ]:
bigz=zz1+zz2
bb=np.real(bigz)

In [ ]:
np.argmax(np.abs(bigz))

In [ ]:
plt.plot(np.real(bigz))
plt.xlim(31000-4096,31000+4096)

In [ ]:
np.angle(zz2[31000])*4096/(2*np.pi)/1845

In [ ]:
zeropts=np.where(np.abs(np.angle(bigz))<1e-2)[0]

In [ ]:
zeropts[100:]

In [ ]:
amp2=np.sum(bb*xcorrtime1)/np.sum(bb**2)

f=plt.gcf()
f.set_size_inches(10,8)
f.set_facecolor('white')
plt.subplot(121)
plt.plot(np.unwrap(np.angle(xcorrtime2[:,0])))
plt.title("Unwrapped coarse phase")
plt.text(0,50,'slope inferred from\n this stretch')
plt.text(0,200,'Original input: k=1839.2, dk=0.025')
plt.text(0,170,'dk inferred from slope = 0.1999\n (expected 0.2)')

plt.subplot(122)
plt.title('Random 20 sample zoom-in')
plt.plot(amp2*bb,label='interpolated coarse')
plt.plot(xcorrtime1,label='raw/original xcorr')
plt.legend()
plt.xlim(29000,29020)
plt.xticks(rotation=30)
plt.tight_layout()

In [ ]:
plt.title('Random 20 sample zoom-in')
plt.plot(amp2*bb,label='interpolated coarse')
plt.plot(xcorrtime1,label='raw/original xcorr')
plt.legend()
plt.xlim(29400,29450)
plt.xticks(rotation=30)

In [ ]:
-np.angle(zz1[31000])*4096/1835.2/(2*np.pi)

In [ ]:
8-31000/4096

In [ ]:
(8-31000.3/4096)*4096

In [ ]:
xx=mycorr(p0,p1)

In [ ]:
plt.plot(np.real(xcorrtime2))
# plt.plot()

In [ ]:
plt.plot(np.angle(xcorrtime2))